# *RMS Titanic* dataset in Neo4j - Preprocessing

This notebook is a prototype for a data processing pipeline. The two basic steps required to prepare the dataset for import into Neo4j are a cleaning step and a geoparsing step relying on Natural Language Processing (NLP).

This notebook requires a running Elasticsearch container with a geonames index:
```bash
python -m spacy download en_core_web_lg
docker pull elasticsearch:5.5.2
wget https://s3.amazonaws.com/ahalterman-geo/geonames_index.tar.gz --output-file=wget_log.txt
tar -xzf geonames_index.tar.gz
docker run -d -p 127.0.0.1:9200:9200 -v $(pwd)/geonames_index/:/usr/share/elasticsearch/data elasticsearch:5.5.2
```

In [2]:
import os
import sys
import pandas as pd

# For NLP and geoparsing
import nltk
from mordecai import Geoparser
import pycountry

# Download NLP data for country extraction // make this modular by setting nltk data path
nltk.download('treebank')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('punkt') # Download corpora for GPE extraction
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columns

Using TensorFlow backend.
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/gregory/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/gregory/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/gregory/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [42]:
# Set path and import
sys.path[0] = '../'
url = 'https://query.data.world/s/xjk6hp7t7w3553bfpkfshr2bjd67a4'
raw = sys.path[0] + 'data/raw/titanic.csv'
data = pd.read_csv(url)

## Cleaning

In [43]:
# String cleaning and some simple feature engineering
def clean_data(data):
    
    # Calculate total size of family (same surname) including passenger
    data['family.size'] = data['sibsp'] + data['parch'] + 1

    # Add surname column to easily identify relatives. Neo4j can 
    # build relationships based on matching surname and family size.
    data['surname'] = data['name'].str.split(',', expand=True)[0]
    
    # Fill NaN values for Cabins
    #data['cabin'] = data['cabin'].fillna('Unspecified Cabin')

    # Extract deck from cabin number
    data['deck'] = data['cabin'].str[:1]
    #data['deck'].fillna('Unspecified Deck', inplace=True)
    
    # Fill incorrect NaN values of embarked with correct values for passengers
    data.loc[data['ticket'] == '113572', 'embarked'] = 'S'
    
    # Replace embarked with location name
    embarked = {"S": "Southampton", "C": "Cherbourg", "Q": "Queenstown"}
    data['embarked'] = data['embarked'].map(embarked)
    
    # Replace NaN values with Unknown in destination column
    data['home.dest'].fillna("Unspecified Destination", inplace=True)
    
    return data

In [44]:
# Run
data = clean_data(data)

In [45]:
# Save. Careful not to overwrite data after lengthy ML steps.
#data.to_csv('../data/clean/titanic_clean.csv', index=False)

## Geoparsing

Geoparsing the data involves three of steps:
1. Remapping abbreviations with full names.
2. Extracting the country from the unstructured text in *home.dest*.
3. Converting the parsed ISO country values into names.

In [46]:
# Create dictionary of Country:[strings]
# If row.`home.dest` is in map assign Country value

abbrev_map = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    "Alberta": "AB",
    "British Columbia": "BC",
    "Manitoba": "MB",
    "New Brunswick": "NB",
    "Newfoundland": "NL",
    "Northwest Territories": "NT",
    "Nova Scotia": "NS",
    "Nunavut": "NU",
    "Ontario": "ON",
    "Prince Edward Island": "PE",
    "Quebec": "PQ",
    "Saskatchewan": "SK",
    "Yukon": "YT",
    "Northern Ireland": "NI"}

# Invert mappings
abbrev_map = {v: k for k, v in abbrev_map.items()}

# Apply mappings to replace state abbreviations
#data['home.dest'] = data['home.dest'].str.split().apply(lambda x: ' '.join([abbrev_map.get(word, word) for word in x]))

# Refactor as function
def remap_abbrev(series):
    remapper = lambda row: ' '.join([abbrev_map.get(word, word) for word in row])
    series = series.str.split().apply(remapper)
    return series

In [47]:
# Test
data['home.dest'] = remap_abbrev(data['home.dest'])

In [48]:
# Inspect
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,family.size,surname,deck
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,Southampton,2,NaN,"St Louis, Missouri",1.0,Allen,B
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,Southampton,11,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,135.0,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C


In [49]:
# Can be used with Pandas apply method. Use batch version for better speed.
def extract_country(row):
    geo = Geoparser(country_threshold=0.9)
    inferred = geo.geoparse(row)
    country_range = range(len(inferred))
    home_countries = set([inferred[i]['country_predicted'] for i in country_range])
    home_countries = ", ".join(home_countries)
    
    return home_countries

In [50]:
# batch_geoparse uses nlp.pipe to speed things up
def batch_extract_country(series):
    countries = []
    geo = Geoparser()
    batch = geo.batch_geoparse(series)
    for doc_list in batch:
        row = ", ".join(set([entry['country_predicted'] for entry in doc_list]))
        countries.append(row)
    
    return pd.Series(countries)

In [51]:
# Converts country ISO abbreviation to English name
def lookup_country_name(row):
    if row == "":
        return ""
    else:
        words = row.split(', ')
        lookup = lambda country: pycountry.countries.lookup(country).name
        names = list(map(lookup, words))
        names = ", ".join(names)
        return names

In [52]:
# Reduce size (optional, takes less time)
data = data[:5]

# Geoparses home.dest to infer destination countries for each passenger. Outputs ISO abbreviation.
# This step takes a while depending on the machine.
data['home.country'] = batch_extract_country(data['home.dest'])

100%|██████████| 5/5 [00:01<00:00,  3.26it/s]


In [53]:
# Inspect
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,family.size,surname,deck,home.country
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,Southampton,2,NaN,"St Louis, Missouri",1.0,Allen,B,USA
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,Southampton,11,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,CAN
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,CAN
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,135.0,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,CAN
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,CAN


In [39]:
# Converts ISO country code to country name. Having the country name makes a better label for the node.
data['home.country'] = data['home.country'].apply(lookup_country_name)

In [41]:
# Inspect
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,family.size,surname,deck,home.country
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,Southampton,2,NaN,"St Louis, Missouri",1.0,Allen,B,United States
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,Southampton,11,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,Canada
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,Canada
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,135.0,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,Canada
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,Southampton,NaN,NaN,"Montreal, Quebec / Chesterville, Ontario",4.0,Allison,C,Canada


In [9]:
# Save final dataset
#data.to_csv(sys.path[0] + 'data/clean/titanic_final.csv', index=False)